In [7]:
import cv2
import numpy as np
import pandas as pd
import os
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def calculate_angle(p1, p2, p3):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    angle = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
    )
    return np.degrees(angle)

def normalize_data(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

# פונקציה לחישוב ציון תנועה על פי הנוסחה שניתנה
def calculate_movement_score(velocity_avg, angle_score, confidence_score):
    return 0.4 * velocity_avg + 0.3 * angle_score + 0.3 * confidence_score

def initialize_chunk_data():
    """איתחול מילון נתונים חדש לצ'אנק"""
    return {
        "left_shoulder_velocities": [],
        "left_shoulder_confidences": [],
        "right_shoulder_velocities": [],
        "right_shoulder_confidences": [],
        "left_elbow_velocities": [],
        "left_elbow_angles": [],
        "left_elbow_confidences": [],
        "right_elbow_velocities": [],
        "right_elbow_angles": [],
        "right_elbow_confidences": [],
        "left_hip_velocities": [],
        "left_hip_confidences": [],
        "right_hip_velocities": [],
        "right_hip_confidences": [],
        "left_knee_velocities": [],
        "left_knee_angles": [],
        "left_knee_confidences": [],
        "right_knee_velocities": [],
        "right_knee_angles": [],
        "right_knee_confidences": []
    }

def calculate_chunk_scores(chunk_data):
    """חישוב ציוני תנועה לצ'אנק"""
    # חישוב ממוצעים וציונים עבור כל חלקי גוף
    
    # כתף שמאל - רק מהירות וביטחון
    left_shoulder_velocity_avg = np.mean(chunk_data["left_shoulder_velocities"]) if chunk_data["left_shoulder_velocities"] else 0
    left_shoulder_confidence_avg = np.mean(chunk_data["left_shoulder_confidences"]) if chunk_data["left_shoulder_confidences"] else 0
    
    # כתף ימין - רק מהירות וביטחון
    right_shoulder_velocity_avg = np.mean(chunk_data["right_shoulder_velocities"]) if chunk_data["right_shoulder_velocities"] else 0
    right_shoulder_confidence_avg = np.mean(chunk_data["right_shoulder_confidences"]) if chunk_data["right_shoulder_confidences"] else 0
    
    # מרפק שמאל - מהירות, זווית וביטחון
    left_elbow_velocity_avg = np.mean(chunk_data["left_elbow_velocities"]) if chunk_data["left_elbow_velocities"] else 0
    left_elbow_angle_avg = np.mean(chunk_data["left_elbow_angles"]) if chunk_data["left_elbow_angles"] else 0
    left_elbow_confidence_avg = np.mean(chunk_data["left_elbow_confidences"]) if chunk_data["left_elbow_confidences"] else 0
    
    # מרפק ימין - מהירות, זווית וביטחון
    right_elbow_velocity_avg = np.mean(chunk_data["right_elbow_velocities"]) if chunk_data["right_elbow_velocities"] else 0
    right_elbow_angle_avg = np.mean(chunk_data["right_elbow_angles"]) if chunk_data["right_elbow_angles"] else 0
    right_elbow_confidence_avg = np.mean(chunk_data["right_elbow_confidences"]) if chunk_data["right_elbow_confidences"] else 0
    
    # ירך שמאל - רק מהירות וביטחון
    left_hip_velocity_avg = np.mean(chunk_data["left_hip_velocities"]) if chunk_data["left_hip_velocities"] else 0
    left_hip_confidence_avg = np.mean(chunk_data["left_hip_confidences"]) if chunk_data["left_hip_confidences"] else 0
    
    # ירך ימין - רק מהירות וביטחון
    right_hip_velocity_avg = np.mean(chunk_data["right_hip_velocities"]) if chunk_data["right_hip_velocities"] else 0
    right_hip_confidence_avg = np.mean(chunk_data["right_hip_confidences"]) if chunk_data["right_hip_confidences"] else 0
    
    # ברך שמאל - מהירות, זווית וביטחון
    left_knee_velocity_avg = np.mean(chunk_data["left_knee_velocities"]) if chunk_data["left_knee_velocities"] else 0
    left_knee_angle_avg = np.mean(chunk_data["left_knee_angles"]) if chunk_data["left_knee_angles"] else 0
    left_knee_confidence_avg = np.mean(chunk_data["left_knee_confidences"]) if chunk_data["left_knee_confidences"] else 0
    
    # ברך ימין - מהירות, זווית וביטחון
    right_knee_velocity_avg = np.mean(chunk_data["right_knee_velocities"]) if chunk_data["right_knee_velocities"] else 0
    right_knee_angle_avg = np.mean(chunk_data["right_knee_angles"]) if chunk_data["right_knee_angles"] else 0
    right_knee_confidence_avg = np.mean(chunk_data["right_knee_confidences"]) if chunk_data["right_knee_confidences"] else 0
    
    # חישוב ציוני תנועה עבור ברכיים ומרפקים (לקביעת דומיננטיות)
    left_elbow_score = calculate_movement_score(left_elbow_velocity_avg, left_elbow_angle_avg, left_elbow_confidence_avg)
    right_elbow_score = calculate_movement_score(right_elbow_velocity_avg, right_elbow_angle_avg, right_elbow_confidence_avg)
    left_knee_score = calculate_movement_score(left_knee_velocity_avg, left_knee_angle_avg, left_knee_confidence_avg)
    right_knee_score = calculate_movement_score(right_knee_velocity_avg, right_knee_angle_avg, right_knee_confidence_avg)
    
    # עבור איברים ללא זווית (כתפיים וירכיים), נשתמש בערך זווית 0 לצורך הנוסחה
    left_shoulder_score = calculate_movement_score(left_shoulder_velocity_avg, 0, left_shoulder_confidence_avg)
    right_shoulder_score = calculate_movement_score(right_shoulder_velocity_avg, 0, right_shoulder_confidence_avg)
    left_hip_score = calculate_movement_score(left_hip_velocity_avg, 0, left_hip_confidence_avg)
    right_hip_score = calculate_movement_score(right_hip_velocity_avg, 0, right_hip_confidence_avg)
    
    # חישוב ציון כללי כממוצע של כל הציונים מכל האיברים
    overall_score = (
        left_elbow_score + right_elbow_score + 
        left_knee_score + right_knee_score + 
        left_shoulder_score + right_shoulder_score + 
        left_hip_score + right_hip_score
    ) / 8
    
    # קביעת איבר דומיננטי
    dominant_knee = "left" if left_knee_score > right_knee_score else "right"
    dominant_elbow = "left" if left_elbow_score > right_elbow_score else "right"
    
    return {
        "overall_movement_score": overall_score,
        "left_knee_score": left_knee_score,
        "right_knee_score": right_knee_score,
        "dominant_knee": dominant_knee,
        "left_elbow_score": left_elbow_score,
        "right_elbow_score": right_elbow_score,
        "dominant_elbow": dominant_elbow
    }

def update_dataset_with_chunk_scores(dataset, video_id, start_frame, end_frame, chunk_scores, chunk_index):
    """עדכון ציוני הצ'אנק בדאטאסט"""
    for data_entry in dataset:
        if (data_entry["video_id"] == video_id and 
            start_frame <= data_entry["frame"] <= end_frame):
            data_entry["chunk_index"] = chunk_index
            data_entry["overall_movement_score"] = chunk_scores["overall_movement_score"]
            data_entry["left_knee_score"] = chunk_scores["left_knee_score"]
            data_entry["right_knee_score"] = chunk_scores["right_knee_score"]
            data_entry["dominant_knee"] = chunk_scores["dominant_knee"]
            data_entry["left_elbow_score"] = chunk_scores["left_elbow_score"]
            data_entry["right_elbow_score"] = chunk_scores["right_elbow_score"]
            data_entry["dominant_elbow"] = chunk_scores["dominant_elbow"]

video_folder = "final_videos"
video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])

dataset = []
selected_keypoints = [11, 12, 13, 14, 23, 24, 25, 26]  # כתף, מרפק, ירך, ברך לשני הצדדים

MIN_X = -1
MAX_X = 1
MIN_Y = -1
MAX_Y = 1
MIN_VELOCITY = -10
MAX_VELOCITY = 10
MIN_ACCELERATION = -10
MAX_ACCELERATION = 10

# שינוי עיקרי: נחשב ציונים כל 60 פריימים במקום פר סרטון
FRAMES_PER_CHUNK = 60

for video_index, video_file in enumerate(video_files):
    video_id = video_index + 1
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    prev_keypoints = None
    prev_velocity = None
    prev_time = None
    frame_index = 0
    chunk_index = 0
    
    # איתחול מילון לנתוני תנועה לצ'אנק הנוכחי
    current_chunk_data = initialize_chunk_data()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # אם הגענו לסוף צ'אנק (60 פריימים), נחשב ציוני תנועה עבור הצ'אנק ונאתחל מחדש
        if frame_index > 0 and frame_index % FRAMES_PER_CHUNK == 0:
            # חישוב ציוני תנועה לצ'אנק הנוכחי
            chunk_scores = calculate_chunk_scores(current_chunk_data)
            
            # הוספת ציוני הצ'אנק לכל השורות בדאטאסט בטווח הפריימים של הצ'אנק
            chunk_start = frame_index - FRAMES_PER_CHUNK
            chunk_end = frame_index - 1
            update_dataset_with_chunk_scores(dataset, video_id, chunk_start, chunk_end, chunk_scores, chunk_index)
            
            # איתחול מחדש של נתוני הצ'אנק
            current_chunk_data = initialize_chunk_data()
            chunk_index += 1

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        if results.pose_landmarks:
            height, width, _ = frame.shape
            keypoints = results.pose_landmarks.landmark

            current_keypoints = []
            current_scores = []
            for idx in range(len(keypoints)):
                kp = keypoints[idx]
                current_keypoints.append([kp.x * width, kp.y * height])
                current_scores.append(kp.visibility)

            current_keypoints = np.array(current_keypoints)
            current_scores = np.array(current_scores)

            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

            velocity = None
            acceleration = None

            if prev_keypoints is not None and prev_time is not None:
                time_diff = current_time - prev_time
                if time_diff > 0:
                    velocity = (current_keypoints - prev_keypoints) / time_diff
                    if prev_velocity is not None:
                        acceleration = (velocity - prev_velocity) / time_diff

            angles = {
                25: ("left_knee_angle", calculate_angle(current_keypoints[23], current_keypoints[25], current_keypoints[27])),
                26: ("right_knee_angle", calculate_angle(current_keypoints[24], current_keypoints[26], current_keypoints[28])),
                13: ("left_elbow_angle", calculate_angle(current_keypoints[11], current_keypoints[13], current_keypoints[15])),
                14: ("right_elbow_angle", calculate_angle(current_keypoints[12], current_keypoints[14], current_keypoints[16]))
            }

            data_entry = {
                "video_id": video_id,
                "frame": frame_index,
                "chunk_index": chunk_index
            }

            for i in selected_keypoints:
                point = current_keypoints[i]
                score = current_scores[i]
                norm_x = normalize_data(point[0], MIN_X, MAX_X)
                norm_y = normalize_data(point[1], MIN_Y, MAX_Y)
                data_entry[f"keypoint_{i}_x"] = norm_x
                data_entry[f"keypoint_{i}_y"] = norm_y
                data_entry[f"keypoint_{i}_confidence"] = score  # visibility

                # איסוף נתונים לצ'אנק הנוכחי
                if i == 11:  # כתף שמאל
                    current_chunk_data["left_shoulder_confidences"].append(score)
                elif i == 12:  # כתף ימין
                    current_chunk_data["right_shoulder_confidences"].append(score)
                elif i == 23:  # ירך שמאל
                    current_chunk_data["left_hip_confidences"].append(score)
                elif i == 24:  # ירך ימין
                    current_chunk_data["right_hip_confidences"].append(score)

                if i in angles:
                    angle_name, angle_value = angles[i]
                    data_entry[angle_name] = angle_value
                    
                    # איסוף זוויות לצ'אנק הנוכחי
                    if i == 13:  # מרפק שמאל
                        current_chunk_data["left_elbow_angles"].append(angle_value)
                        current_chunk_data["left_elbow_confidences"].append(score)
                    elif i == 14:  # מרפק ימין
                        current_chunk_data["right_elbow_angles"].append(angle_value)
                        current_chunk_data["right_elbow_confidences"].append(score)
                    elif i == 25:  # ברך שמאל
                        current_chunk_data["left_knee_angles"].append(angle_value)
                        current_chunk_data["left_knee_confidences"].append(score)
                    elif i == 26:  # ברך ימין
                        current_chunk_data["right_knee_angles"].append(angle_value)
                        current_chunk_data["right_knee_confidences"].append(score)

                if velocity is not None:
                    vel_magnitude = np.linalg.norm(velocity[i])
                    norm_vx = normalize_data(velocity[i][0], MIN_VELOCITY, MAX_VELOCITY)
                    norm_vy = normalize_data(velocity[i][1], MIN_VELOCITY, MAX_VELOCITY)
                    data_entry[f"keypoint_{i}_velocity_x"] = norm_vx
                    data_entry[f"keypoint_{i}_velocity_y"] = norm_vy
                    
                    # איסוף מהירויות לצ'אנק הנוכחי
                    if i == 11:  # כתף שמאל
                        current_chunk_data["left_shoulder_velocities"].append(vel_magnitude)
                    elif i == 12:  # כתף ימין
                        current_chunk_data["right_shoulder_velocities"].append(vel_magnitude)
                    elif i == 13:  # מרפק שמאל
                        current_chunk_data["left_elbow_velocities"].append(vel_magnitude)
                    elif i == 14:  # מרפק ימין
                        current_chunk_data["right_elbow_velocities"].append(vel_magnitude)
                    elif i == 23:  # ירך שמאל
                        current_chunk_data["left_hip_velocities"].append(vel_magnitude)
                    elif i == 24:  # ירך ימין
                        current_chunk_data["right_hip_velocities"].append(vel_magnitude)
                    elif i == 25:  # ברך שמאל
                        current_chunk_data["left_knee_velocities"].append(vel_magnitude)
                    elif i == 26:  # ברך ימין
                        current_chunk_data["right_knee_velocities"].append(vel_magnitude)

                if acceleration is not None:
                    norm_ax = normalize_data(acceleration[i][0], MIN_ACCELERATION, MAX_ACCELERATION)
                    norm_ay = normalize_data(acceleration[i][1], MIN_ACCELERATION, MAX_ACCELERATION)
                    data_entry[f"keypoint_{i}_acceleration_x"] = norm_ax
                    data_entry[f"keypoint_{i}_acceleration_y"] = norm_ay

            dataset.append(data_entry)
            prev_keypoints = current_keypoints.copy()
            prev_velocity = velocity.copy() if velocity is not None else None
            prev_time = current_time

        frame_index += 1

    # אם נשארו פריימים בסוף הסרטון שלא הושלמו לצ'אנק שלם, נחשב גם עבורם
    if frame_index % FRAMES_PER_CHUNK != 0 and any(len(val) > 0 for val in current_chunk_data.values()):
        chunk_scores = calculate_chunk_scores(current_chunk_data)
        chunk_start = chunk_index * FRAMES_PER_CHUNK
        chunk_end = frame_index - 1
        update_dataset_with_chunk_scores(dataset, video_id, chunk_start, chunk_end, chunk_scores, chunk_index)

    cap.release()

df = pd.DataFrame(dataset)
df.to_csv("mediapipe_motion_dataset_with_windows_scores.csv", index=False)
print("✅ סיום עיבוד הסרטונים, הנתונים נשמרו בקובץ mediapipe_motion_dataset_with_windows_scores.csv")

✅ סיום עיבוד הסרטונים, הנתונים נשמרו בקובץ mediapipe_motion_dataset_with_windows_scores.csv
